In [ ]:
import pandas as pd
import numpy as np
import os
import grid2op
from grid2op.Chronics import ChangeNothing
from chronix2grid.kpi.Generator_parameter_checker import EnergyMix_AprioriChecker

In [ ]:
input_path = os.path.join("..", "example", "input", "generation", "case118_l2rpn_wcci")

In [ ]:
df = pd.read_csv(os.path.join(input_path, "prods_charac.csv"))

In [ ]:
df

In [ ]:
pmaxs = df.groupby(["type"])["Pmax"].sum()
types = pmaxs.index.to_numpy()
pmaxs = pmaxs.to_numpy()

In [ ]:
types, pmaxs, pmaxs.sum()

In [ ]:
def get_apriori_energy_mix(capacity_factor, average_load):
  apriori_energy_mix = capacity_factor * pmaxs / average_load
  total = np.nansum(apriori_energy_mix)
  if total > 100:
    print("WARNING: apriori energy mix w/o thermal exceeds average load!")
    apriori_energy_mix[0] -= (total - 100)
    apriori_energy_mix[3] = 0
  else:
    apriori_energy_mix[3] = 100 - total
  return apriori_energy_mix


In [ ]:
capacity_factor = np.array([30, 95, 15, np.nan, 25])
average_load = 2800
capacity_factor_df = pd.DataFrame(data=capacity_factor, columns=['capacity_factor'], index=types)

apriori_energy_mix = get_apriori_energy_mix(capacity_factor, average_load)

In [ ]:
grid_path = os.path.join(input_path, "grid.json")

In [ ]:
env118_withoutchron = grid2op.make(
    input_path,
    test=True,
    grid_path=grid_path, # assign it the 118 grid
    chronics_class=ChangeNothing, # tell it to change nothing (not the most usable environment...)
)

In [ ]:
Target_EM_percentage=pd.DataFrame(data=[18, 40, 4, 30, 6], columns=['target_energy_mix'], index=types)

#Variable used to anticipate the energy mix a priori. Update them after chronics generation if too different
PeakLoad = 4200 #expected peak load

Capacity_df=EnergyMix_AprioriChecker(env118_withoutchron, Target_EM_percentage, PeakLoad, average_load, capacity_factor_df)
Capacity_df["Apriori_energy_mix"]

In [ ]:
apriori_energy_mix